In [ ]:
import pandas as pd
import os
import seaborn as sns
from ast import literal_eval
from matplotlib import pyplot as plt

In [ ]:
exp_path = "/mnt/e/Experiment_Toy_2"
alpha = 0.1

In [ ]:
df_cov_inter = pd.read_csv(os.path.join(exp_path, "cond_cov_interactions.csv"), sep=";")

In [ ]:
df_cov_inter

In [ ]:
df_select = df_cov_inter.loc[(df_cov_inter['Methode'] == "jackknife_plus_lin_lasso_None") & (
            df_cov_inter['Strategie'] == "Random_3") & (df_cov_inter['Run'] == "run_3")]

In [ ]:
df_select['Inter'] = df_select['Inter'].apply(literal_eval)
df_select = df_select.explode('Inter')

In [ ]:
df_select = df_select.reset_index(drop=True)
df_select["cov_x_int"]= df_select["Inter"]/(df_select["Cov"])

In [ ]:
features = df_cov_inter.Feature.unique().tolist()
features

In [ ]:
features_select = [x for x in features if "_+_" not in x]

In [ ]:
features_select.append("no_8x8dct_+_no_fast_pskip")
features_select.append("no_fast_pskip_+_no_weightb")

In [ ]:
features_select

In [ ]:
df_select_plot = df_select.loc[df_select['Feature'].isin(features_select)]

In [ ]:
df_select_plot

In [ ]:
plt.figure(figsize=(6, len(features_select) * 0.3075))


ax1 = sns.boxplot(data=df_select_plot, x='Cov', y='Feature', hue='Feature')
#sns.move_legend(ax1, "upper left", bbox_to_anchor=(1, 1))
ax1.axvline(x=(1 - 2 * alpha), color='r', linestyle='--')
ax1.axvline(x=(1 - alpha), color='y', linestyle='--')
#sns.move_legend(ax1, "upper left", bbox_to_anchor=(1, 0.97))
# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
ax3 = sns.kdeplot(df_select.loc[df_select['Feature'].isin(features_select)], x='Inter', hue='Feature')
sns.move_legend(ax3, "upper left", bbox_to_anchor=(1, 1))

In [ ]:
plt.figure(figsize=(6, len(features_select) * 0.34))
ax3 = sns.boxplot(df_select.loc[df_select['Feature'].isin(features_select)], x='Inter',y='Feature', hue='Feature')
#sns.move_legend(ax3, "upper left", bbox_to_anchor=(1, 0.97))
ax3.set_xlim(left=0)
#plt.tight_layout(rect=[0, 0, 1, 0.95])

In [ ]:
# Abbildung mit Subplots erstellen
fig, axs = plt.subplots(1, 2, figsize=(12, 4.6), sharey=True)  # Horizontale Subplots

# Boxplot für 'Cov'
sns.boxplot(data=df_select_plot, x='Cov',y='Feature', hue='Feature', ax=axs[0])
axs[0].axvline(x=(1 - 2 * alpha), color='r', linestyle='--')
axs[0].axvline(x=(1 - alpha), color='y', linestyle='--')

# KDE-Plot für 'Inter'
#sns.kdeplot(data=df_select.loc[df_select['Feature'].isin(features_select)], x='Inter', hue='Feature', ax=axs[1])

# Boxplot für 'Inter'
sns.boxplot(data=df_select.loc[df_select['Feature'].isin(features_select)], x='Inter', y='Feature', hue='Feature', ax=axs[1])
axs[1].set_xlim(left=0)

# Gemeinsame Legende erstellen
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='center left', bbox_to_anchor=(1, 0.55), prop={'size': 8.3})



# Layout anpassen und anzeigen
plt.tight_layout()
plt.show()

In [ ]:
# Abbildung mit Subplots erstellen
fig, axs = plt.subplots(1, 2, figsize=(12, 4.6))  # Horizontale Subplots

# Boxplot für 'Cov'
#sns.boxplot(data=df_select_plot, x='Cov', hue='Feature', ax=axs[0])
#axs[0].axvline(x=(1 - 2 * alpha), color='r', linestyle='--')
#axs[0].axvline(x=(1 - alpha), color='y', linestyle='--')

# KDE-Plot für 'Inter'
sns.kdeplot(data=df_select.loc[df_select['Feature'].isin(features_select)], x='Inter', hue='Feature', ax=axs[0])

# Boxplot für 'Inter'
sns.boxplot(data=df_select.loc[df_select['Feature'].isin(features_select)], x='Inter', y='Feature', hue='Feature', ax=axs[1])
axs[1].set_xlim(left=0)

# Gemeinsame Legende erstellen
handles, labels = axs[1].get_legend_handles_labels()
fig.legend(handles, labels, loc='center left', bbox_to_anchor=(0, 0.55), prop={'size': 8.3})



# Layout anpassen und anzeigen
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(3, 0.75*len(features_select) * 0.34))
g = sns.violinplot(data=df_select.loc[df_select['Feature'].isin(features_select)], x='Inter', y='Feature', hue='Feature')
g.set_xlabel('Intervallbreite')
g.set_ylabel('Features')
plt.savefig(os.path.join(exp_path, "interv-features.png"), bbox_inches='tight')


In [ ]:
plt.figure(figsize=(3, 0.75*len(features_select) * 0.34))
g = sns.violinplot(data=df_select.loc[df_select['Feature'].isin(features_select)], x='cov_x_int', y='Feature', hue='Feature')
g.set_xlabel('Intervallbreite/Coverage')
g.set_ylabel('Features')
plt.savefig(os.path.join(exp_path, "cov-x-interv-features.png"), bbox_inches='tight')

In [ ]:
df_select.loc[df_select['Feature'].isin(features_select)]

In [ ]:
numeric_cols = df_select.select_dtypes(include='number').columns
numeric_cols

In [ ]:
df = df_select
df['Cov'] = pd.to_numeric(df['Cov'], errors='coerce')
df['Inter'] = pd.to_numeric(df['Inter'], errors='coerce')
df['cov_x_int'] = pd.to_numeric(df['cov_x_int'], errors='coerce')
numeric_cols = df.select_dtypes(include='number').columns

In [ ]:
grouped_df = df.groupby('Feature')[numeric_cols].mean().reset_index()

In [ ]:
grouped_df.loc[grouped_df['Feature'].isin(features_select)]

In [ ]:
grouped_df['Feature'] = pd.Categorical(grouped_df['Feature'], categories=features_select, ordered=True)
sorted_df = grouped_df.loc[grouped_df['Feature'].isin(features_select)].sort_values('Feature').reset_index(drop=True)
sorted_df

# Statistische Test

In [ ]:
df = df_select.loc[df_select['Feature'].isin(features_select)]

In [ ]:
from scipy.stats import kruskal
groups = df.groupby('Feature')['cov_x_int'].apply(list)

# Liste aller Features ohne 'root'
features = [feature for feature in groups.index if feature != 'root']

# Resultate speichern
results = []

# Kruskal-Wallis-Test für jedes Feature gegen 'root'
for feature in features:
    stat, p_value = kruskal(groups['root'], groups[feature])
    results.append({'Feature': feature, 'p_value': p_value})

# Ergebnisse als DataFrame
results_df = pd.DataFrame(results)

# Ausgabe der Ergebnisse
print(results_df)

In [ ]:
results_df

In [ ]:
results_df['Feature'] = pd.Categorical(results_df['Feature'], categories=features_select, ordered=True)
sorted_df = results_df.loc[results_df['Feature'].isin(features_select)].sort_values('Feature').reset_index(drop=True)

In [ ]:
sorted_df

In [ ]:
i = 0
for index, row in sorted_df.iterrows():
    if row['p_value']> 0.00263:
        i = i +1
print(i)